In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10752,2024-05-14,Taiwan Liga T1,08:00,New Taipei CTBC DEA,Taipei Taishin Mars,1.37,2.93,178.5,1.85,1.85,0.0,0.00,0.00,pzqkSPoS,0.729927,0.341297,0.540541,0.540541,0.071224,155.974,53.232935,0.341294,1.8,1.643168,0.912871,133.65,1.662,0.254008,0.152833,30.0,189.046,48.240026,0.255176,1.8,1.643168,0.912871,152.65,2.196,0.464898,0.211702,19.0,81,71,1.65,2.15,125.090,284.202,0.513064,0.000000,NaN,-2.23,-0.446,-0.829596,0.700944,0.6,-0.100944,0.99,0.198,9.747475,0.375700,0.3,-0.075700
10753,2024-05-14,Suécia Basketligan,14:04,Borås,Norrkoping,2.10,1.70,159.5,1.80,1.86,1.5,1.93,1.55,6LTcLPdL,0.476190,0.588235,0.555556,0.537634,0.064426,202.538,66.909753,0.330357,0.6,1.341641,2.236068,245.05,2.748,1.064716,0.387451,-31.0,106.274,19.230843,0.180955,3.0,0.000000,0.000000,82.50,1.206,0.032863,0.027250,96.0,65,66,3.77,1.25,112.930,121.890,0.148865,0.023184,0.154426,1.43,0.286,3.846154,0.799007,0.9,0.100993,0.55,0.110,6.363636,0.762785,0.8,0.037215
10754,2024-05-14,República Tcheca Nbl,13:00,Decin,Usti n. Labem,1.55,2.34,164.5,1.85,1.85,0.0,0.00,0.00,QkQQc0WO,0.645161,0.427350,0.540541,0.540541,0.072512,197.402,67.425871,0.341566,1.2,1.643168,1.369306,204.00,2.408,0.787921,0.327210,4.0,141.044,37.477262,0.265713,1.8,1.643168,0.912871,122.12,1.744,0.518151,0.297105,-4.0,75,86,2.72,1.42,189.672,223.316,0.287205,0.000000,NaN,4.50,0.900,0.611111,0.657551,0.8,0.142449,-2.69,-0.538,-2.490706,0.506387,0.4,-0.106387
10755,2024-05-14,Portugal Lpb,17:00,CD Póvoa,Oliveirense,2.62,1.45,159.5,1.87,1.79,0.0,0.00,0.00,OhSCxukf,0.381679,0.689655,0.534759,0.558659,0.071335,319.142,344.761846,1.080277,1.2,1.643168,1.369306,234.65,4.164,4.488154,1.077847,-32.0,161.968,111.129205,0.686118,2.4,1.341641,0.559017,98.28,1.922,1.276820,0.664319,38.0,65,78,3.61,1.26,156.960,165.104,0.406543,0.030912,NaN,1.56,0.312,5.192308,0.000000,0.0,0.000000,3.16,0.632,0.712025,0.000000,0.0,0.000000
10756,2024-05-14,Polônia Liga De Basquete,12:45,Anwil Wloclawek,Spojnia Stargard,1.25,3.77,157.5,1.80,1.86,0.0,0.00,0.00,tWHkehje,0.800000,0.265252,0.555556,0.537634,0.065252,115.280,10.868063,0.094275,1.8,1.643168,0.912871,106.08,1.360,0.162019,0.119131,32.0,207.756,63.326823,0.304813,1.2,1.643168,1.369306,202.96,2.626,0.745372,0.283843,-14.0,68,86,1.56,2.36,104.756,233.136,0.709924,0.023184,NaN,0.00,0.000,inf,0.841211,0.7,-0.141211,-1.86,-0.372,-7.446237,0.408510,0.2,-0.208510
10757,2024-05-14,Lituânia Lkl,12:50,Lietkabelis,Juventus,1.37,2.99,165.5,1.87,1.87,0.0,0.00,0.00,pWbPpsHq,0.729927,0.334448,0.534759,0.534759,0.064375,124.986,27.823154,0.222610,1.8,1.643168,0.912871,168.84,1.462,0.343686,0.235079,7.0,253.802,152.454186,0.600682,1.8,1.643168,0.912871,134.75,3.380,2.682844,0.793741,-11.0,84,77,2.01,1.75,134.494,259.466,0.525465,0.000000,NaN,-2.72,-0.544,-0.680147,0.724519,0.5,-0.224519,-0.71,-0.142,-14.014085,0.472082,0.4,-0.072082
10758,2024-05-14,Lituânia Lkl,12:00,Mazeikiai,Zalgiris Kaunas,8.80,1.02,162.5,1.87,1.87,0.0,0.00,0.00,bRrbPubD,0.113636,0.980392,0.534759,0.534759,0.094029,347.686,283.076187,0.814172,0.0,0.000000,NaN,114.21,4.756,4.177204,0.878302,-88.0,110.676,15.925454,0.143893,1.8,1.643168,0.912871,97.52,1.120,0.129615,0.115728,75.0,81,92,1.41,1.06,139.856,114.452,1.120426,0.000000,NaN,-1.45,-0.290,-26.896552,0.503459,0.5,-0.003459,-1.46,-0.292,-0.068493

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10769,14:45,Israel Superliga,Nes Ziona,Hapoel Beer Sheva,1.57,Back Home
